In [1]:
! pip install  ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.2.0-py3-none-any.whl size=24525 sha256=b7588b953aa7dd609eee8bf109283558e55bcc9af01a97a19ba9c1d7f1a7b52f
  Stored in directory: /tmp/pip-ephem-wheel-cache-htirotq3/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.2.0
    Uninstalling boston311-0.2.0:
      Successfully uninstalled boston311-0.2.0


In [2]:
import os
import pandas as pd
import numpy as np
import pickle
from ast import literal_eval
from datetime import datetime, timedelta
from kerastuner import HyperParameters
from boston311 import Boston311LogReg, Boston311KerasNN

Using TensorFlow backend


2023-11-01 23:02:04.114950: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-01 23:02:04.161403: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-01 23:02:04.162116: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-01 23:02:04.833968: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_548799/3798184988.py:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperParameters


In [3]:
today_datestring, tomorrow_datestring, thirty_days_ago_datestring = Boston311LogReg().get_datestrings()

In [4]:
KerasNN_model = Boston311KerasNN()

In [5]:
KerasNN_model.train_date_range={'start':'2020-01-01','end':thirty_days_ago_datestring}

KerasNN_model.predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring}

KerasNN_model.feature_columns=['queue', 'subject', 'reason', 'department']

KerasNN_model.scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']}, 'survivalTimeMin':300, 'survivalTimeFill':tomorrow_datestring}

KerasNN_model.epochs = 20
KerasNN_model.batch_size = 32

In [6]:

hour_interval = 48
max_days = 120

#KerasNN_model.bin_edges = KerasNN_model.generate_time_bins_statistics(df, num_intervals=60)
KerasNN_model.bin_edges = KerasNN_model.generate_time_bins_fixed_interval(hour_interval, max_days)

start_nodes = 256  
end_nodes = 128
#l2_0 = 0.00001
#learning_rate = 7.5842e-05
l2_0 = 0.001
learning_rate = 0.0001

hp = HyperParameters()
hp.Fixed('start_nodes', start_nodes)
hp.Fixed('end_nodes', end_nodes)
hp.Fixed('l2_0', l2_0)
hp.Fixed('learning_rate', learning_rate)
bin_number = len(KerasNN_model.bin_edges) - 1
hp.Fixed('final_layer', bin_number)
hp.Fixed('final_activation', 'softmax')
KerasNN_model.best_hyperparameters = hp

In [7]:


pickle_file = 'dataframe.pkl'
xz_file = '../dataframe.csv.xz'
#unpack xz file if it exists
if os.path.exists(xz_file):
    os.system(f'unxz {xz_file}')
    df = pd.read_csv('dataframe.csv')
    pickle.dump(df, open(pickle_file, "wb"))
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_three_cols.csv'

df = KerasNN_model.pkl_load_data(EXTRA_mydata_FILE, pickle_file)

# if df has a column service_request_id, do the following
if 'service_request_id' in df.columns:
    df.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)

    df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
    is_numeric = df['case_enquiry_id'].str.isnumeric()
    df = df[is_numeric]
    df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')
    for col in ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']:
        df[col] = df[col].apply(literal_eval).apply(np.array)

    pickle.dump(df, open(pickle_file, "wb"))

# Assuming df is your DataFrame
column_names = ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']
df = KerasNN_model.flatten_and_replace_columns(df, column_names)



In [8]:
case_data_file = 'case_data.pkl'
case_data_csv = 'all_311_cases.csv'
data = KerasNN_model.pkl_load_data(case_data_csv, case_data_file)

In [9]:
#get current datetime in Boston timezone as string
my_datetime_str = KerasNN_model.get_current_datetime_str()

In [10]:
KerasNN_model.run_pipeline(data, df)


bin_labels is None
Starting Training at 2023-11-01 23:02:37.501231
input_dim: 1020
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               261376    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 61)                7869      
                                                                 
Total params: 302141 (1.15 MB)
Trainable params: 302141 (1.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
<class 'pandas.core.frame.DataFrame'> (168320, 61)
<class 'pandas.core.frame.DataFrame'> (42081, 61)
run fit

Epoch 1/20


2023-11-01 23:02:40.941474: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1373491200 exceeds 10% of free system memory.


5247/5260 [============================>.] - ETA: 0s - loss: 1.4473 - accuracy: 0.7288 - top_k_categorical_accuracy: 0.8219

2023-11-01 23:02:52.490896: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 686753760 exceeds 10% of free system memory.


5260/5260 [==============================] - 13s 2ms/step - loss: 1.4469 - accuracy: 0.7288 - top_k_categorical_accuracy: 0.8219 - val_loss: 1.6117 - val_accuracy: 0.6603 - val_top_k_categorical_accuracy: 0.7560
Epoch 2/20
5260/5260 [==============================] - 11s 2ms/step - loss: 1.2584 - accuracy: 0.7408 - top_k_categorical_accuracy: 0.8300 - val_loss: 1.5054 - val_accuracy: 0.6732 - val_top_k_categorical_accuracy: 0.7625
Epoch 3/20
5260/5260 [==============================] - 11s 2ms/step - loss: 1.1970 - accuracy: 0.7437 - top_k_categorical_accuracy: 0.8323 - val_loss: 1.4745 - val_accuracy: 0.6685 - val_top_k_categorical_accuracy: 0.7616
Epoch 4/20
5260/5260 [==============================] - 11s 2ms/step - loss: 1.1618 - accuracy: 0.7451 - top_k_categorical_accuracy: 0.8335 - val_loss: 1.4496 - val_accuracy: 0.6716 - val_top_k_categorical_accuracy: 0.7634
Epoch 5/20
5260/5260 [==============================] - 11s 2ms/step - loss: 1.1387 - accuracy: 0.7461 - top_k_categori

0.7489603161811829

In [11]:
predictions = KerasNN_model.predict(df)

Checking files_dict
files_dict is None
trying to call get311URLs
trying to get csv URLs


Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmpt_2g32xl.csv for year 2023
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmpfm8veglw.csv for year 2022
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmp88p9g82n.csv for year 2021
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv for year 2020
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv for year 2019
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/tmp7602cia8.csv for year 2018
Found URL:

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:331: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


columns in data before drop: Index(['case_enquiry_id', 'open_dt', 'sla_target_dt', 'closed_dt', 'on_time',
       'case_status', 'closure_reason', 'case_title', 'subject', 'reason',
       'type', 'queue', 'department', 'submitted_photo', 'closed_photo',
       'location', 'fire_district', 'pwd_district', 'city_council_district',
       'police_district', 'neighborhood', 'neighborhood_services_district',
       'ward', 'precinct', 'location_street_name', 'location_zipcode',
       'latitude', 'longitude', 'geom_4326', 'source', 'survival_time',
       'event', 'ward_number', 'survival_time_hours'],
      dtype='object')
columns to drop: Index(['case_status', 'case_title', 'city_council_district', 'closed_dt',
       'closed_photo', 'closure_reason', 'fire_district', 'geom_4326',
       'latitude', 'location', 'location_street_name', 'location_zipcode',
       'longitude', 'neighborhood', 'neighborhood_services_district',
       'on_time', 'open_dt', 'police_district', 'precinct', 'pwd_

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:262: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, fake_df], ignore_index=True)


484/484 [==============================] - 0s 580us/step


In [12]:
print(predictions)

[[1.56276435e-01 3.10001597e-02 2.35506669e-02 ... 5.43830311e-03
  6.58557983e-03 2.89393395e-01]
 [7.40468860e-01 1.40885875e-01 4.92025875e-02 ... 2.92857021e-05
  5.07032746e-05 2.27548589e-04]
 [3.52099001e-01 1.69185564e-01 6.14025295e-02 ... 1.78430998e-03
  2.51933932e-03 1.30301211e-02]
 ...
 [7.66762137e-01 1.35053545e-01 4.26015295e-02 ... 2.13420208e-05
  3.69925765e-05 2.02469877e-04]
 [7.65175879e-01 1.40136063e-01 4.23399620e-02 ... 1.51481190e-05
  2.79881806e-05 1.14482405e-04]
 [7.60507941e-01 1.37570754e-01 4.60410602e-02 ... 1.85126919e-05
  3.36519442e-05 1.38173375e-04]]


In [13]:
#set model folder constant
MODEL_FOLDER = './daily_models'

def save_model_to_dir(model, folder_name, datetime_string=None):
    dir_path = os.path.join(MODEL_FOLDER, folder_name)
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    if datetime_string is None:
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    else:
        timestamp = datetime_string
    
    model_name = timestamp + "_" + model.model_type
    properties_name = model_name
    
    model.save(dir_path, model_name, properties_name)


save_model_to_dir(KerasNN_model, KerasNN_model.model_type, datetime_string=my_datetime_str)


/home/briarmoss/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
